Below is the recreation of a GSP First Reaction Method, following lecture slides. First the neccesary variables are initiated. The possible random values are first limited to 0, 1 and will be adjusted later.

In [1]:
import numpy as np
from fractions import Fraction
import matplotlib.pyplot as plt

# Labels and couples all possible events and rates and stores in an array
E_1 = 100 # number of people in area 1
E_2 = 20 # number of people in area 2

k_1 = 0.1 # Rate of people moving from area 1 to 2
k_2 = 0.1 # Rate of people moving from area 2 to 1

t = 0 # Sets and initiates current time variable at 0

# Creates an empty dictionary for event timesteps
event_dy = {}

Now, we write a function to determine the time until a given event occurs.

In [149]:
def gsp_occurrence(k_m, k_n):
    """ This function first calculates rate of occurence of given event and calculates the timestep 
        of the occurence by generating exponentially from a uniform random number according to 
        an exponential distribution with mean 1/R_m """

    # Timestep of occurence
    dt_m = np.random.exponential(1/k_m) 
    dt_n = np.random.exponential(1/k_n)
    return dt_m, dt_n

# Passes current event state and event rate constant to function
dt_1, dt_2 = gsp_occurrence(k_1, k_2)

# Stores the timesteps into the dictionary and sorts in order from occurence
event_dy["E_1"] = dt_1
event_dy["E_2"] = dt_2
sort_event_dy = dict(sorted(event_dy.items(), key = lambda item: item [1]))

# Checks which event occurs first
first_event_key, first_event_value = list(sort_event_dy.items())[0]


We now want to adjust the state of the passed event after it occurs and take into account the state of the other event. We also want to save the timestep at which the occurence takes place and update the current time. Only the first event takes place. 

In [ ]:
# Updates current time
#t += dt

# Appends the timestep to the list
#events_1.append(t)

E_1, E_2

def gsp_update(m, n):
    """ This function updates the given event by removing 1 from pool m and adding 1 to pool n. """
    m -= 1
    n += 1
    return m, n

#E_1, E_2 = gsp_update(E_1, E_2)

# Checks which event occurs and calls on appropriate function
if first_event_key == "E_1": # people move from 1 to 2
    E_1, E_2 = gsp_update(E_1, E_2)
    t += dt_1

elif first_event_key == "E_2": # People move from 2 to 1
    E_2, E_1 = gsp_update(E_2, E_1)
    t += dt_2

# reset the dictionary
event_dy = {}
E_1, E_2

The GSP First Reaction Method will now be implemented with SIR. First all initial values are set.

In [2]:
# Initiates SIR valuables and parameters
S = 90          # Susceptible population   
I = 10          # Infected population
R = 0           # Recovered population

beta = 0.1      # Infection rate
gamma = 0.1     # Recovery rate
mu = 0.05       # Natural birth and death rate

t = 0           # Current time
t_total = 100   # Time period over which gsp takes place

sir = [S, I, R, S + I + R]

A function is initiated to change the SIR values according to the first event as given in the function above. We want to execute these functions not just once but loop over them during a given time period. We initiate a function that loops over t while it remains within this period.

In [3]:
def sir_update(S, I, R, N, key):

    if key == 'infection':
        S -= 1
        I += 1
        return S, I, R, N
    elif key == 'recovery':
        R += 1
        I -= 1
        return S, I, R, N
    elif key == 'birth':
        S += 1
        N += 1
        return S, I, R, N
    elif key == 'death S':
        S -= 1
        N -= 1
        return S, I, R, N
    elif key == 'death I':
        I -= 1
        N -= 1
        return S, I, R, N
    elif key == 'death R':
        R -= 1
        N -= 1
        return S, I, R, N


A function to determine the gsp of sir values

In [4]:
def gsp_sir(t_total, sir, beta, gamma, mu):
    """
   
    """
    S, I, R, N = sir
    t = 0
    sir_array = ([S, I, R, N, t])

    while t <= t_total:
        events = {}

        # calculates rates of change and adds all events into the dictionary

        if S > 0 and I > 0 and N > 0:
            dt_infection = np.random.exponential(1/(beta*S*I/N))
            events['infection'] = dt_infection

        if I > 0:
            dt_recovery = np.random.exponential(1/(gamma*I))
            events['recovery'] = dt_recovery

        if N > 0:
            dt_birth = np.random.exponential(1/(mu*N))
            events['birth'] = dt_birth

        if S > 0:
            dt_death_S = np.random.exponential(1/(mu*S))
            events['death S'] = dt_death_S

        if I > 0:
            dt_death_I = np.random.exponential(1/(mu*I))
            events['death I'] = dt_death_I

        if R > 0:
            dt_death_R = np.random.exponential(1/(mu*R))
            events['death R'] = dt_death_R
    

        # Sorts the events in the dictionary
        sorted_events = dict(sorted(events.items(), key = lambda item: item [1]))

        #Assigns variable to the first event in the dictionary
        first_event_key, first_event_time = list(sorted_events.items())[0]

        # Updates current time
        t += first_event_time

        S, I, R, N = sir_update(S, I, R, N, first_event_key)
        
        sir_array.append([S, I, R, N, t])

    return sir_array

Implements the gsp and plots the data.

In [9]:
def plot_gsp(sir_array):
    
    #sir_array = np.array(sir_array)

    S = sir_array[:, 0]
    I = sir_array[:, 1]
    R = sir_array[:, 2]
    t = sir_array[:, 3]

    plt.figure(figsize=(6, 4))
    plt.plot(t, S, label='Susceptible (S)', color='blue')
    plt.plot(t, I, label='Infected (I)', color='orange')
    plt.plot(t, R, label='Recovered (R)', color='green')

    plt.xlabel('Time')
    plt.ylabel('Population')
    plt.title('Disease Simulation Over Time')
    plt.legend()
    plt.grid(True)
    plt.show()

    return

sir_array = gsp_sir(t_total, sir, beta, gamma, mu)
sir_array
#plot_gsp(sir_array)

[90,
 10,
 0,
 100,
 0,
 [91, 10, 0, 101, 0.18765626890059828],
 [92, 10, 0, 102, 0.21757488218646093],
 [93, 10, 0, 103, 0.3486390550610066],
 [94, 10, 0, 104, 0.4068194631129549],
 [94, 9, 0, 103, 0.4705858970202853],
 [95, 9, 0, 104, 0.5315093297167196],
 [94, 9, 0, 103, 0.6626399066772385],
 [93, 9, 0, 102, 0.6968908818927664],
 [92, 9, 0, 101, 0.9476120570305708],
 [93, 9, 0, 102, 1.1010444331313227],
 [94, 9, 0, 103, 1.1154888495469053],
 [93, 10, 0, 103, 1.1499726080719175],
 [94, 10, 0, 104, 1.1620020558636834],
 [95, 10, 0, 105, 1.4411392333738986],
 [94, 10, 0, 104, 1.6445826291323697],
 [93, 10, 0, 103, 1.9441549225940487],
 [92, 10, 0, 102, 1.9939357585274655],
 [93, 10, 0, 103, 2.015967178028745],
 [94, 10, 0, 104, 2.124679075290494],
 [93, 10, 0, 103, 2.1991637817239553],
 [94, 10, 0, 104, 2.203390865877379],
 [95, 10, 0, 105, 2.222275967893651],
 [94, 10, 0, 104, 2.3032539315887477],
 [94, 9, 1, 104, 2.387369947144813],
 [93, 9, 1, 103, 2.3899468834810964],
 [94, 9, 1, 1